In [18]:
import json
from pprint import pprint
import pandas as pd
import os
import re
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math

import random
from pathlib import Path
from transliterate import translit


In [19]:
# Получаем список файлов в папки
ds = pd.read_json('2108.json')
# ds = pd.read_json('1108.json')

In [20]:
# PATH = 'G:/JN/sarawan/sber.json'
data = ds.copy()

In [21]:
ds.columns

Index(['_id', 'name_store', 'main_category_name', 'main_category_img_url',
       'category_name', 'category_img_url', 'category_permalink',
       'product_name', 'product_img_link', 'product_instamart_price',
       'product_original_unit_price', 'product_price', 'product_human_volume',
       'product_items_per_pack', 'product_price_type', 'product_stock',
       'product_stock_rate', 'product_max_stock_rate', 'product_discounted',
       'product_discount', 'product_unit_price', 'product_description',
       'product_description_original', 'product_properties'],
      dtype='object')

In [24]:
sorted(ds['name_store'].unique())

['5ka',
 'auchansm',
 'dixy',
 'fixpricefd',
 'globus',
 'magnit_express',
 'metro',
 'okey',
 'perekrestok',
 'selgros',
 'verniy_fd',
 'victoria']

In [67]:
shops_dict = {
    'magnit_express': 'Магнит',
    'lenta': 'Лента',
    'fixpricefd': 'Фикспрайс',
    'metro': 'Метро',
    'victoria': 'Виктория',
    'auchan': 'Ашан',
    'okey': 'Окей',
    'globus': 'Глобус',
    'dixy': 'Дикси',
    'verniy_fd': 'Верный',
    'auchansm': 'Ашан',
    'perekrestok': 'Перекресток',
    'selgros': "Селгрос",
    'verniy_fd': "Верный",
    '5ka': 'Пятерочка',
    'victoria': "Виктория" }   
     

In [26]:
#Запись словарей в файл    
with open('shops_dict', 'w', encoding='utf-8') as f:
    f.write(json.dumps(shops_dict))

In [27]:
# Чтение словаря из файла
with open('shops_dict', 'r') as f:
    shops_dict = json.loads(f.read())

In [32]:
def give_me_dataset(data):
    
    # regex for all nums and len(word)>3 -> new column product_group_name
    df = data.copy()
    df.loc[(df['product_price_type'] == 'per_kilo'), 'product_price'] = df['product_unit_price']
    df.loc[(df['product_price_type'] == 'per_kilo'), 'product_human_volume'] = '1 кг'
    df =  df[(df['main_category_name'] != 'Алкоголь')]
    
      
#     list_of_names = df.product_name.apply(lambda x: sorted(re.compile("(%s|%s)" % (re1, re2)).findall(str(x.lower())))).tolist()
#     list_of_weight = df.product_human_volume.apply(lambda x: re.findall(re1, x)).tolist()
#     small_name_col = []
#     for num, w_or_d in enumerate(list_of_names):
# #         if len(w_or_d) <= 3 & (re.search(re1, str(w_or_d)) is None):
#         if (re.search(re1, str(w_or_d)) is None):
#             weight = list_of_weight[num]
#             small_name  = str(weight[0]) + ' ' + ' '.join(w_or_d)
#         else:
#             small_name = ' '.join(w_or_d)
#         small_name_col.append(small_name)
    
#     df.insert(loc=0, column='product_group_name', value=small_name_col)   

    
    df.reset_index(inplace=True)
    
    # Создание новой колонки для группировки продуктов
    
    re1 = r"\d+" # nums
    re2 = '\w{3,30}' # words (цифры , буквы и др символы)
    re11 = '[a-zа-яё]+\d+|\d+[a-zа-яё]+|\d+\,*\d+\%|\w[а-яё,a-z]{2,30}' #буквы, цифры с буквами   #'\w[а-яё,a-z]{2,30}\d*'   # words
    re3 = r"\d+\s\b[млкгшт]{1,2}\b|\d+\,*\d+\s\b[млкгшт]{1,2}" 
    
    list_of_names = []
#     list_of_names_orig = []
    for num, i in enumerate(df.product_name): 
        try:
            weight = re.findall(re3, df.product_human_volume[num])[0].split()
            if (weight[0] == '1' and weight[1] == 'шт'):
                name = ' '.join(sorted(re.compile("(%s|%s)" % (re1, re2)).findall(i.lower())))
#                 name = ' '.join(sorted(re.compile("(%s)" % re2).findall(i.lower())))
#                 name_orig = df.product_name[num].lower()              
            elif len(weight) == 2 and (weight[1] == 'л' or weight[1] == 'мл'):
                name = str(weight[0]) + ' ' + ' '.join(sorted(re.compile("(%s)" % re11).findall(i.lower()))) 
                df.product_price_type[num] = 'per_volume'
            else:
                name = str(weight[0]) + ' ' + ' '.join(sorted(re.compile("(%s)" % re11).findall(i.lower()))) 
#                 name_orig = str(weight[0]) + ' ' + ' '.join(re.compile("(%s)" % re11).findall(i.lower()))
        except:
            name = ' '.join(sorted(re.compile("(%s)" % re11).findall(i.lower())))
#             name_orig = ' '.join(re.compile("(%s)" % re11).findall(i.lower()))

        list_of_names.append(name)
#         list_of_names_orig.append(name_orig)
    df.insert(loc=0, column='product_group_name', value=list_of_names) 
#     df.insert(loc=0, column='product_group_name_orig', value=list_of_names_orig) 

    
#     df['product_group_name'] = df['product_group_name'].apply(lambda x: x.replace(' metro chef', ''))
    df['product_group_name'] = df['product_group_name'].apply(lambda x: x.replace(' metro chef', ''))
    df['product_group_name'] = df['product_group_name'].apply(lambda x: x.replace(' бзмж', ''))
#     df['product_group_name_orig'] = df['product_group_name_orig'].apply(lambda x: x.replace(' metro chef', ''))
#     df['product_group_name_orig'] = df['product_group_name_orig'].apply(lambda x: x.replace(' бзмж', ''))
    
    
    # Уберем категорию продукты
    df.loc[df['main_category_name'] == 'Продукты',  'main_category_name'] = df['category_name']
    
    df.loc[df['main_category_name'] == 'Чай и кофе', 'main_category_name'] = 'Чай, кофе, какао'
    df.loc[df['main_category_name'] == 'Чай, кофе', 'main_category_name'] = 'Чай, кофе, какао'
    df.loc[df['main_category_name'] == 'Канцелярия, книги', 'main_category_name'] = 'Канцтовары, книги, творчество'
    df.loc[df['main_category_name'] == 'Сыры', 'main_category_name'] = 'Молоко, сыр, яйца, растительные продукты'
    df.loc[df['main_category_name'] == 'Молочные продукты, яйца', 'main_category_name'] = 'Молоко, сыр, яйца, растительные продукты'
    df.loc[df['main_category_name'] == 'Молочные продукты, сыры, яйца', 'main_category_name'] = 'Молоко, сыр, яйца, растительные продукты'
    df.loc[df['main_category_name'] == 'Рыба, морепродукты', 'main_category_name'] = 'Рыба, морепродукты, икра'
    df.loc[df['main_category_name'] == 'Сладости', 'main_category_name'] = 'Сладости, снеки'
    df.loc[df['main_category_name'] == 'Растительные масла', 'main_category_name'] = 'Растительные масла, специи, соусы'


    # delete duplicates in each shop and make clear_dataset 
    clear_dataset = pd.DataFrame() 
    for shop in df['name_store'].unique():
        df_temp = df[df['name_store'] == shop]
        df_temp.drop_duplicates(subset='product_group_name', inplace=True)
        try:
            df_temp.loc[df_temp['name_store'] == shop, 'name_store'] = shops_dict.get(shop, [])
        except:
            print(shop, ' - new shop!')
            
        
        clear_dataset = pd.concat([clear_dataset, df_temp])
        
        
    
#     pka_df = df[df['name_store'] == '5ka']
#     pka_df.drop_duplicates(subset='product_group_name', inplace=True)
#     pka_df.loc[pka_df['name_store'] == '5ka', 'name_store'] = 'Пятеречка' 
    
#     perekrestok_df = df[df['name_store'] == 'perekrestok']
#     perekrestok_df.drop_duplicates(subset='product_group_name', inplace=True)
#     perekrestok_df.loc[perekrestok_df['name_store'] == 'perekrestok', 'name_store'] = 'Перекресток'  
    
#     selgros_df = df[df['name_store'] == 'selgros']
#     selgros_df.drop_duplicates(subset='product_group_name', inplace=True)
#     selgros_df.loc[selgros_df['name_store'] == 'selgros', 'name_store'] = 'Селгрос' 
    
#     magnit_express_df = df[df['name_store'] == 'magnit_express']
#     magnit_express_df.drop_duplicates(subset='product_group_name', inplace=True)
#     magnit_express_df.loc[magnit_express_df['name_store'] == 'magnit_express', 'name_store'] = 'Магнит'
    
#     lenta_df = df[df['name_store'] == 'lenta']
#     lenta_df.drop_duplicates(subset='product_group_name', inplace=True)
#     lenta_df.loc[lenta_df['name_store'] == 'lenta', 'name_store'] = 'Лента'
    
#     fixprice_df = df[df['name_store'] == 'fixpricefd']
#     fixprice_df.drop_duplicates(subset='product_group_name', inplace=True)
#     fixprice_df.loc[fixprice_df['name_store'] == 'fixpricefd', 'name_store'] = 'Фикспрайс'
    
#     metro_df = df[df['name_store'] == 'metro']
#     metro_df.drop_duplicates(subset='product_group_name', inplace=True)
#     metro_df.loc[metro_df['name_store'] == 'metro', 'name_store'] = 'Метро'
    
#     victoria_df = df[df['name_store'] == 'victoria']
#     victoria_df.drop_duplicates(subset='product_group_name', inplace=True)
#     victoria_df.loc[victoria_df['name_store'] == 'victoria', 'name_store'] = 'Виктория'
    
#     auchan_df = df[df['name_store'] == 'auchansm']
#     auchan_df.drop_duplicates(subset='product_group_name', inplace=True)
#     auchan_df.loc[auchan_df['name_store'] == 'auchan', 'name_store'] = 'Ашан'
    
#     okey_df = df[df['name_store'] == 'okey']
#     okey_df.drop_duplicates(subset='product_group_name', inplace=True)
#     okey_df.loc[okey_df['name_store'] == 'okey', 'name_store'] = 'Окей'
    
#     globus_df = df[df['name_store'] == 'globus']
#     globus_df.drop_duplicates(subset='product_group_name', inplace=True)
#     globus_df.loc[globus_df['name_store'] == 'globus', 'name_store'] = 'Глобус'
    
    
#     globus_df = df[df['name_store'] == 'dixy']
#     globus_df.drop_duplicates(subset='product_group_name', inplace=True)
#     globus_df.loc[globus_df['name_store'] == 'dixy', 'name_store'] = 'Дикси'
    
    
#     globus_df = df[df['name_store'] == 'verniy_fd']
#     globus_df.drop_duplicates(subset='product_group_name', inplace=True)
#     globus_df.loc[globus_df['name_store'] == 'verniy_fd', 'name_store'] = 'Верный'
    
    
#     clear_dataset = pd.concat([magnit_express_df,lenta_df,metro_df,victoria_df,auchan_df,okey_df,globus_df, fixprice_df, pka_df, perekrestok_df, selgros_df])
    
    # delete inedible goods
    clear_dataset.drop(clear_dataset[clear_dataset.product_human_volume == '50 м'].index, inplace=True)
    
#     clear_dataset.drop(clear_dataset[clear_dataset.product_human_volume.str.contains('см')|
#                                      clear_dataset.product_human_volume.str.contains('лист')|
#                                      clear_dataset.product_human_volume.str.contains('Вт')|
#                                      clear_dataset.product_human_volume.str.contains('Гб')|
#                                      clear_dataset.product_human_volume.str.contains('пак')|
#                                      clear_dataset.product_human_volume.str.contains('уп')].index, inplace=True)
    
#     clear_dataset.drop(clear_dataset[clear_dataset.main_category_name.str.contains('Электрон')|
#                                      clear_dataset.main_category_name.str.contains('емонт')|
# #                                      clear_dataset.main_category_name.str.contains('Канцелярия')|
#                                      clear_dataset.main_category_name.str.contains('гигиена')|
#                                      clear_dataset.main_category_name.str.contains('Товары для дома')|
#                                      clear_dataset.main_category_name.str.contains('обувь')|
#                                      clear_dataset.main_category_name.str.contains('химия')|
#                                      clear_dataset.main_category_name.str.contains('Авто')|
#                                      clear_dataset.main_category_name.str.contains('Дача')|
#                                      clear_dataset.main_category_name.str.contains('екстиль')|
#                                      clear_dataset.main_category_name.str.contains('книги')|
#                                      clear_dataset.main_category_name.str.contains('порт')|
#                                      clear_dataset.main_category_name.str.contains('-10% со СберПрайм')|
#                                      clear_dataset.main_category_name.str.contains('Спецпредложения')|
#                                      clear_dataset.main_category_name.str.contains('Рекомендац')|
#                                      clear_dataset.main_category_name.str.contains('Алкоголь')].index, inplace=True)
    
#     clear_dataset.drop(clear_dataset[clear_dataset.product_name.str.contains('play')|
#                                      clear_dataset.product_name.str.contains('Play')|
#                                      clear_dataset.product_name.str.contains('игурка')|
#                                      clear_dataset.product_name.str.contains('пластилин')|
#                                      clear_dataset.product_name.str.contains('песок')|
#                                      clear_dataset.product_name.str.contains('онструктор')|
#                                      clear_dataset.product_name.str.contains('Пенал')|
#                                      clear_dataset.product_name.str.contains('Краски')|
#                                      clear_dataset.product_name.str.contains('Салфетки')|
#                                      clear_dataset.product_name.str.contains('Ванночка')].index, inplace=True)
    
#     clear_dataset.drop(clear_dataset[clear_dataset.category_name.str.contains('увениры')|
#                                      clear_dataset.category_name.str.contains('пятновыводители')|
#                                      clear_dataset.category_name.str.contains('пароварки')|
#                                      clear_dataset.category_name.str.contains('оловные уборы')|
#                                      clear_dataset.category_name.str.contains('езодорант')|
#                                      clear_dataset.category_name.str.contains('ирлянд')|
#                                      clear_dataset.category_name.str.contains('ксессуар')|
#                                      clear_dataset.category_name.str.contains('дом')|
#                                      clear_dataset.category_name.str.contains('игиена')|
#                                      clear_dataset.category_name.str.contains('Авто')|
#                                      clear_dataset.category_name.str.contains('игры')|
#                                      clear_dataset.category_name.str.contains('музыка')|
#                                      clear_dataset.category_name.str.contains('уход')|
#                                      clear_dataset.category_name.str.contains('Зубн')|
# #                                      clear_dataset.category_name.str.contains('Игруш')|
#                                      clear_dataset.category_name.str.contains('Купание')|
#                                      clear_dataset.category_name.str.contains('переноск')|
#                                      clear_dataset.category_name.str.contains('антисептик')|
#                                      clear_dataset.category_name.str.contains('Наполнител')|
#                                      clear_dataset.category_name.str.contains('Носки')|
#                                      clear_dataset.category_name.str.contains('посуда')|
#                                      clear_dataset.category_name.str.contains('рабочего')|
#                                      clear_dataset.category_name.str.contains('Ортопед')|
#                                      clear_dataset.category_name.str.contains('Пазлы')|
#                                      clear_dataset.category_name.str.contains('упаковка')|
#                                      clear_dataset.category_name.str.contains('Презервативы')|
#                                      clear_dataset.category_name.str.contains('Присыпка')|
#                                      clear_dataset.category_name.str.contains('Прогулка')|
#                                      clear_dataset.category_name.str.contains('Пустышки')|
#                                      clear_dataset.category_name.str.contains('декор')|
# #                                      clear_dataset.category_name.str.contains('салфетки')|
# #                                      clear_dataset.category_name.str.contains('Шампуни')|
#                                      clear_dataset.category_name.str.contains('Школьные')|
#                                      clear_dataset.category_name.str.contains('Шарики')|
#                                      clear_dataset.category_name.str.contains('игруш')|
#                                      clear_dataset.category_name.str.contains('Подгузники')].index, inplace=True)

#     Создание колонки 'total_weight_kg' - вес продукта
    volume_list = clear_dataset['product_human_volume'].apply(lambda x: x.replace(',', '.').split()).to_list()
    new_list = []
#     re3 = r"\d+[ ]\w{1,2}"
#     re4 = r"\d+\,*\d+\s+\[гкглмл]{1,2}\b"

    """Изменила шаблон регулярного выражения""" 

    re3 = r"\d+\,*\d+\s\b[млкг]{1,2}\b"
    re4 = r"\d+\,*\d+\s"

    for num, i in enumerate(volume_list):
        if len(i) == 4:
            volume_list[num] = str(float(i[0]) * float(i[3])) + ' ' + i[1]
        if len(i) == 2 and i[1] == 'шт':
            try:
                '''Исправлена ошибка new_df на clear_dataset'''
                volume_list[num] = re.compile("(%s)" % (re3)).findall(str(clear_dataset.product_name.to_list()[num]))[0].split()
                volume_list[num][0] = volume_list[num][0].replace(',', '.')  
            except:
                volume_list[num] = '1 шт'
            
    for num, i in enumerate(volume_list):
        if len(i) == 2:
            if i[1] == 'г' or i[1] == 'мл':
                new_list.append(round(float(i[0]) / 1000, 2))
            elif i[1] == 'мг':
                new_list.append(round(float(i[0]) / 1000000, 2))
            elif i[1] == 'л':
                new_list.append(round(float(i[0]), 2))
            elif i[1] == 'шт':
                new_list.append(0)
                flag[num] = 1
            elif i[1] == 'кг':
                new_list.append(round(float(i[0])))
            else:
                new_list.append(0)
        else:
            new_list.append(0)
            
    
    discount = [round((1 - (i / j)) * 100) for i, j in zip(clear_dataset.product_unit_price.to_list(), clear_dataset.product_original_unit_price.to_list())]
    
    clear_dataset.drop(['product_discount'], axis=1, inplace=True)
    clear_dataset.insert(loc=20, column='product_discount', value=discount)   
    clear_dataset.insert(loc=0, column='total_weight_kg', value=new_list)
    clear_dataset['slug_product_name'] = clear_dataset['product_name'].apply(lambda x: '-'.join(translit(x, language_code='ru', reversed=True).lower().replace("\'", '').replace(",", '-').split(' ')))
    clear_dataset['slug_main_category_name'] = clear_dataset['main_category_name'].apply(lambda x: '-'.join(translit(x, language_code='ru', reversed=True).lower().replace("\'", '').replace(",", '').split(' ')))
    clear_dataset['slug_category_name'] = clear_dataset['category_name'].apply(lambda x: '-'.join(translit(x, language_code='ru', reversed=True).lower().replace("\'", '').replace(",", '').split(' ')))



    
    for i in range(0, math.ceil(clear_dataset.shape[0]/10000)):
        rg1 = int(f'{i}0000')
        rg2 = int(f'{i+1}0000')
        with open(f'data/sber_{i}.json', 'w', encoding='utf-8') as file:
            clear_dataset[rg1:rg2].to_json(file, force_ascii=False, orient='records')
    
    
    
    return clear_dataset

In [33]:
%%time
new_df = give_me_dataset(data)

<ipython-input-32-76112c3b504b>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.product_price_type[num] = 'per_volume'
<ipython-input-32-76112c3b504b>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.drop_duplicates(subset='product_group_name', inplace=True)
<ipython-input-32-76112c3b504b>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.drop_duplicates(subset='product_group_name', inplace=True)
<ipython-input-32

Wall time: 9min 6s


In [68]:
new_df.loc[new_df['name_store'] == "Фикспрайс", 'main_category_name'].unique()

array(['Товары для дома', 'Канцтовары, книги, творчество',
       'Одежда, обувь, аксессуары', 'Товары для животных',
       'Детские товары', 'Косметика, гигиена', 'Бытовая химия, уборка',
       'Консервы, соленья', 'Хлеб, выпечка', 'Чай, кофе, сахар',
       'Вода, напитки', 'Бакалея', 'Сладости, жвачки',
       'Чипсы, снеки, семечки', 'Колбасные изделия', 'Молочные продукты'],
      dtype=object)

In [69]:
new_df[new_df["product_name"].str.lower().str.contains("картофель молодой".lower())][['product_price_type', 'total_weight_kg', ]]

,product_price_type,total_weight_kg
100260,per_kilo,1.0
34541,per_kilo,1.0
13132,per_package,5.0
104520,per_kilo,1.0
103807,per_package,2.0
109959,per_kilo,1.0


In [70]:
new_df.loc[new_df['product_price_type'] == 'per_volume', 'product_human_volume'].head(5)

436    250 мл
438       1 л
440    500 мл
441    500 мл
448    350 мл
Name: product_human_volume, dtype: object

In [36]:
# до обработки
len(new_df['product_name'].unique())

79879

In [37]:
# С колонкой product_group_name
len(new_df['product_group_name'].unique())

78136

In [32]:
# После приведения слов к начальной форме (см ниже)
len(new_df['product_group_name_prep'].unique())

45434

In [22]:
new_df[['product_human_volume', 'product_group_name', 'product_name']].tail(50)

,product_human_volume,product_group_name,product_name
138359,1 шт,120 воде дезодорант для обуви радуга серебряной,Дезодорант для обуви Радуга на серебряной воде...
138360,400 мл,400 colgate plax ассортименте для ополаскивате...,Ополаскиватель для полости рта Colgate Plax 40...
138361,80 мл,80 biomed вайт зубная комплекс паста,Зубная паста Biomed Вайт комплекс 80 мл
138362,130 мл,130 silcamed ассортименте зубная кореи паста с...,Зубная паста Silcamed Секреты Кореи 130 мл в а...
138363,1 шт,home time для запасные листов одежды ролики,Запасные ролики Home Time для одежды 80 листов...
138364,520 мл,520 dent vilsen ассортименте для ополаскивател...,Ополаскиватель для полости рта Vilsen Dent 520...
138365,45 мл,45 proff ассортименте зубной порошок уголь,Зубной порошок Уголь Proff 45 мл в ассортименте
138366,1 шт,foot happy амортизирующие стельки,Стельки амортизирующие Happy Foot
138367,100 мл,100 aquafresh ассортименте зубная паста,Зубная паста Aquafresh 100 мл в ассортименте
138368,2 шт,2 charcoal colgate density high ассортименте з...,Зубная щетка Colgate High Density Charcoal 1 +...


#### ПРИВЕДЕНИЕ СЛОВ К НАЧАЛЬНОЙ ФОРМЕ

In [30]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer


new_df['product_group_name_prep'] = new_df['product_group_name']

sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_prod_name(txt):
     
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

new_df['product_group_name_prep'] = new_df['product_group_name_prep'].apply(preprocess_prod_name)


In [31]:
new_df['product_group_name_prep'].head(50)

89937                     disney бумажный пакет подарочный
89938        255 315 ламинировать матовый пакет подарочный
89939        коллекционный лицензия микс наклейка объёмный
89947                          lorirocket косметичка пенал
89948    officespace заточить карандаш простой черногра...
89960        114 146 ламинировать матовый пакет подарочный
89997              биджи клетка лист общий скрепка тетрадь
89998               биджи линия лист общий скрепка тетрадь
90029    berlingo fuze tribase ассортимент ручка синий ...
90030       ассортимент ручка синий стамм тропик шариковый
90031                      ассортимент одежда ролик чистка
90032                              губка обувь цена чёрный
90034                  berlingo ассортимент ручной точилка
90035                    bic cristal ручка синий шариковый
90056                            ассортимент линейка стамм
90069                          berlingo fuze карандаш клей
90104    120 360 арт ассортимент бутылка дизайн пакет п.

In [122]:
new_df.loc[new_df['total_weight_kg'] == 0.05, 'product_name'].values

array(['Чай черный Richard Lord Grey в пакетиках 2 г х 25 шт',
       'Чай черный Greenfield Golden Ceylon в пакетиках 2 г 25 шт',
       'Чай черный Лисма Крепкий в пакетиках 2 г х 25 шт', ...,
       'Зубная паста Colgate МаксБлеск с отбеливающими пластинами 50 мл',
       'Зубная паста Colgate Максимальная защита от кариеса 50 мл',
       'Крем-хна Fito Косметик Индийская для волос мокко 50 мл'],
      dtype=object)

In [43]:
new_df[new_df["product_name"].str.lower().str.contains("молоко")].sort_values(by='total_weight_kg', ascending=False).groupby(['product_group_name_prep', 'product_name', 'product_human_volume']).size().reset_index(name='counts').head(50)

,product_group_name_prep,product_name,product_human_volume,counts
0,0925 32 молоко ополье ультрапастеризовать,"Молоко 3,2% ультрапастеризованное 925 мл Ополь...","0,925 г",1
1,097 32 молоко ультрапастеризовать,"Молоко 3,2% ультрапастеризованное 970 мл Кажды...","0,97 кг",1
2,1 005 comfort parmalat безлактозной молоко уль...,"Молоко 0,05% ультрапастеризованное 1 л Parmala...",1 кг,1
3,1 005 comfort parmalat безлактозной молоко уль...,"Молоко 0,05% ультрапастеризованное 1 л Parmala...",1 кг,1
4,1 005 elite valio молоко обезжиренный ультрапа...,"Молоко 0,05% обезжиренное ультрапастеризованно...",1 кг,4
5,1 05 dietalat parmalat молоко ультрапастеризовать,"Молоко 0,5% ультрапастеризованное 1 л Parmalat...",1 кг,5
6,1 05 parmalat молоко ультрапастеризовать,"Молоко 0,5% ультрапастеризованное 1 л Parmalat",1 кг,1
7,1 05 parmalat молоко ультрапастеризовать,"Молоко 0,5% ультрапастеризованное 1 л Parmalat...",1 кг,3
8,1 05 rioba молоко ультрапастеризовать,"Молоко 0,5% ультрапастеризованное 1 л Rioba",1 кг,1
9,1 15 line professional молоко ультрапастеризов...,"Молоко 1,5% ультрапастеризованное 1 л ЭкоНива ...",1 л,1
